BUSINESS PROBLEM:

- The problem that the business deals in here is what the customers think about their hotels.
- By looking at the problem in this way, they can rectify what the customers want and what can be done to increase their reach and values.
- The hospitality business works on hearsay, most of the potential customers look at what the previous customers has to say about the place at which they stayed.
- The feedbacks will help the hotels to overcome the shortcomings in different ways possible


ML PROBLEM STATEMENT:

- The problem is a Machine Learning as we have to classify the reviews of the customers into different categories
- Two different application of Machine Learning is invloved in this problem statement:
1. Natural Language Processing
2. Classification

MODEL USED FOR CLASSIFICATION:
1. LOGISTIC REGRESSION( Accuracy of 70.33% was achieved)
2. SGD CLASSIFIER(66.9%)
- SGD CLASSIFIER was discredited in light of Logistic Regression 

- CHANGED HYPERPARAMETERS OF LOGISTIC REGRESSION 

METHOD USED IN NLP:

- DOC2VEC
- VECTORISATION OF THE CORPUS

# IMPORT LIBRARIES

In [1]:
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
from gensim.models import Doc2Vec
from sklearn import utils
import gensim
from gensim.models.doc2vec import TaggedDocument
import re
import gensim.models.doc2vec

C:\Users\suraj\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# TEXT PREPROCESSING 

READING TRAIN AND TEST DATA

In [3]:
train = pd.read_csv('Train-1554810061973.csv')

In [4]:
test = pd.read_csv('Test-1555730055539.csv')

In [5]:
def label_sentences(corpus, label_type):
    """
    Gensim's Doc2Vec implementation requires each document/paragraph to have a label associated with it.
    We do this by using the TaggedDocument method. The format will be "TRAIN_i" or "TEST_i" where "i" is
    a dummy index of the post.
    """
    labeled = []
    for i, v in enumerate(corpus):
        label = label_type + '_' + str(i)
        labeled.append(TaggedDocument(v.split(), [label]))
    return labeled

In [6]:
from sklearn.model_selection import train_test_split
X_train = train.reviewtext
X_test = test.reviewtext
y_train = train.Sentiment
X_train = label_sentences(X_train, 'Train')
X_test = label_sentences(X_test, 'Test')
all_data = X_train + X_test
 


BUILDING A VOCABULARY FOR THE MODEL 

In [7]:
model_dbow = Doc2Vec(dm=0, vector_size=300, negative=5, min_count=1, alpha=0.065, min_alpha=0.065)
model_dbow.build_vocab([x for x in tqdm(all_data)])

100%|██████████| 9755/9755 [00:00<?, ?it/s]


In [8]:
for epoch in range(30):
    model_dbow.train(utils.shuffle([x for x in tqdm(all_data)]), total_examples=len(all_data), epochs=1)
    model_dbow.alpha -= 0.002
    model_dbow.min_alpha = model_dbow.alpha

100%|██████████| 9755/9755 [00:00<?, ?it/s]


VECTORISATION OF THE CORPUS

In [9]:
def get_vectors(model, corpus_size, vectors_size, vectors_type):
    """
    Get vectors from trained doc2vec model
    :param doc2vec_model: Trained Doc2Vec model
    :param corpus_size: Size of the data
    :param vectors_size: Size of the embedding vectors
    :param vectors_type: Training or Testing vectors
    :return: list of vectors
    """
    vectors = np.zeros((corpus_size, vectors_size))
    for i in range(0, corpus_size):
        prefix = vectors_type + '_' + str(i)
        vectors[i] = model.docvecs[prefix]
    return vectors

In [10]:
train_vectors_dbow = get_vectors(model_dbow, len(X_train), 300, 'Train')
test_vectors_dbow = get_vectors(model_dbow, len(X_test), 300, 'Test')



# MODEL BUILDING

BUILDING A LOGISTIC REGRESSION MODEL 

In [11]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(n_jobs=1, C=1e5)
logreg.fit(train_vectors_dbow, y_train)

LogisticRegression(C=100000.0, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [12]:
#PREDICTING ON THE TEST DATA

y_pred = logreg.predict(test_vectors_dbow)

In [13]:
y_pred = pd.DataFrame(y_pred,columns=['Predictions'])

In [14]:
predictions = pd.concat([test.Reviewid,y_pred],axis=1)

In [15]:
#SAVING THE PREDICTIONS AS CSV 

predictions.to_csv(path_or_buf='predictions.csv')

HYPER-PARAMETERS TUNING

In [16]:
from sklearn.model_selection import GridSearchCV

In [17]:
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000] }
clf = GridSearchCV(LogisticRegression(penalty='l2'), param_grid,cv=3)

In [18]:
clf.fit(train_vectors_dbow,y_train)

GridSearchCV(cv=3, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

FINAL MODEL

In [19]:
clf_best = clf.best_estimator_

In [20]:
#PREDICTING ON TEST DATA

predictions1  = clf_best.predict(test_vectors_dbow)

In [21]:
predictions1 = pd.DataFrame(predictions1,columns=['Predictions'])

In [22]:
predictions1 = pd.concat([test.Reviewid,predictions1],axis=1)

In [23]:
predictions1.to_csv(path_or_buf='predictions1.csv')